This code shows CNN model with 4 convolutional layer with 3 fully connected layers

In [1]:
# Library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping



In [2]:
# Standard Normal Variate Function
def snv(input_data):
  
    # Define a new array and populate it with the corrected data  
    output_data = np.zeros_like(input_data)
    for i in range(input_data.shape[0]): 
        # Apply correction
        output_data[i,:] = (input_data[i,:] - np.mean(input_data[i,:])) / np.std(input_data[i,:])
        
    return output_data

In [3]:
# load the dataset
data = pd.read_csv('/content/Acetaminophen_mixture_CNN_Binary.csv', index_col=0)
data = pd.DataFrame(data)
data.drop(['label_name'], axis=1, inplace=True)
data.rename(columns={'Unnamed: 0': 'id'},inplace=True)
y = data['label']
X = data.drop(['label'],axis=1)
X = X.values[:,1:]
X = snv(X)
X = pd.DataFrame(X)
print((X.shape))


(900, 226)


In [4]:
# Train Test Split and reshaping the data
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)
X_train = X_train.values.reshape(X_train.shape[0],X_train.shape[1],1).astype('float32')
X_test= X_test.values.reshape(X_test.shape[0],X_test.shape[1],1).astype('float32')

In [10]:
# Define Model

tf.keras.metrics.RootMeanSquaredError(
    name="root_mean_squared_error", dtype=None
)

n_timesteps = X_train.shape[1] 
n_features  = X_train.shape[2]  
model = keras.Sequential(name="model_conv1D")
model.add(keras.layers.Input(shape=(n_timesteps,n_features)))
model.add(keras.layers.Conv1D(filters=64, kernel_size=7, activation='relu', name="Conv1D_1"))
model.add(keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu', name="Conv1D_2"))
model.add(keras.layers.Conv1D(filters=16, kernel_size=2, activation='relu', name="Conv1D_3"))
model.add(keras.layers.Conv1D(filters=8, kernel_size=2, activation='relu', name="Conv1D_4"))


model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu', name="Dense_1"))
model.add(keras.layers.Dense(16, activation='relu',name="Dense_2"))
model.add(keras.layers.Dense(1, name="Dense_3"))



model.compile(loss ='mse', optimizer='adam',metrics=['mse','mae',tf.keras.metrics.RootMeanSquaredError()])

In [11]:
model.summary()

Model: "model_conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1D_1 (Conv1D)           (None, 220, 64)           512       
                                                                 
 Conv1D_2 (Conv1D)           (None, 216, 32)           10272     
                                                                 
 Conv1D_3 (Conv1D)           (None, 215, 16)           1040      
                                                                 
 Conv1D_4 (Conv1D)           (None, 214, 8)            264       
                                                                 
 flatten_1 (Flatten)         (None, 1712)              0         
                                                                 
 Dense_1 (Dense)             (None, 32)                54816     
                                                                 
 Dense_2 (Dense)             (None, 16)               

In [12]:
history = model.fit(X_train, y_train, epochs=20,verbose=1)

Epoch 1/20
23/23 [==============================] - 2s 35ms/step - loss: 0.1439 - mse: 0.1439 - mae: 0.3313 - root_mean_squared_error: 0.3794
Epoch 2/20
23/23 [==============================] - 1s 33ms/step - loss: 0.0340 - mse: 0.0340 - mae: 0.1481 - root_mean_squared_error: 0.1844
Epoch 3/20
23/23 [==============================] - 1s 42ms/step - loss: 0.0102 - mse: 0.0102 - mae: 0.0811 - root_mean_squared_error: 0.1011
Epoch 4/20
23/23 [==============================] - 2s 67ms/step - loss: 0.0096 - mse: 0.0096 - mae: 0.0763 - root_mean_squared_error: 0.0980
Epoch 5/20
23/23 [==============================] - 1s 61ms/step - loss: 0.0051 - mse: 0.0051 - mae: 0.0543 - root_mean_squared_error: 0.0713
Epoch 6/20
23/23 [==============================] - 2s 72ms/step - loss: 0.0048 - mse: 0.0048 - mae: 0.0528 - root_mean_squared_error: 0.0690
Epoch 7/20
23/23 [==============================] - 1s 60ms/step - loss: 0.0043 - mse: 0.0043 - mae: 0.0491 - root_mean_squared_error: 0.0657
Epoch 

In [13]:
mse = model.evaluate(X_test, y_test)
print('Mean Squared Error!',mse)
print(history.history['mae'])


6/6 [==============================] - 0s 13ms/step - loss: 0.0019 - mse: 0.0019 - mae: 0.0358 - root_mean_squared_error: 0.0439
Mean Squared Error! [0.0019242799608036876, 0.0019242799608036876, 0.03581830859184265, 0.04386661574244499]
[0.33132094144821167, 0.148071750998497, 0.08108583837747574, 0.07634329050779343, 0.054296743124723434, 0.052754078060388565, 0.04909380152821541, 0.04582517594099045, 0.045576732605695724, 0.051299259066581726, 0.04680237919092178, 0.04296451061964035, 0.04185524955391884, 0.04331584647297859, 0.037647005170583725, 0.0375487245619297, 0.03655562549829483, 0.04141583293676376, 0.0427413284778595, 0.03408324345946312]


In [14]:
#Predicting Mystery Data
myst = pd.read_csv('/content/Mystery_testing_data.csv', index_col=0)
myst = pd.DataFrame(myst)
myst_predictions = pd.DataFrame(myst["label_name"])
myst.drop(['label_name'], axis=1, inplace=True)

myst_actual = pd.read_csv('/content/Mystery_labels.csv')
myst_actual = myst_actual['Actual ']
# Applying SNV to myst data
myst = myst.values[:,1:]
myst = snv(myst)
myst = pd.DataFrame(myst)

myst= myst.values.reshape(myst.shape[0],myst.shape[1],1).astype('float32')
preds = model.predict(myst)

se = pd.Series(preds.tolist())
myst_predictions.insert(loc=0, column='predictions', value=se.values)
#Saving results
myst_predictions.to_csv('/content/Mystery_Prediction_1DCNN_data_binary.csv')

# Evaluate model
mse = model.evaluate(myst, myst_actual)
print('Mean Squared Error!',mse)

4/4 [==============================] - 0s 9ms/step - loss: 0.0824 - mse: 0.0824 - mae: 0.1607 - root_mean_squared_error: 0.2871
Mean Squared Error! [0.0824470967054367, 0.0824470967054367, 0.16065822541713715, 0.2871360182762146]


In [16]:
#Predicting Ternary Samples Data
data = pd.read_csv('/content/ternary_samples.csv', index_col=0)
data = pd.DataFrame(data)
ternary_predictions = pd.DataFrame()

ternary_predictions["label_name"] =data["label_name"]
ternary_predictions['Actual'] = data['label']

data.drop(['label_name'], axis=1, inplace=True)
data.rename(columns={'Unnamed: 0': 'id'},inplace=True)

y = data['label']
X = data.drop(['label'],axis=1)
X = X.values[:,1:]
X = snv(X)
X = pd.DataFrame(X)

ternary= X.values.reshape(X.shape[0],X.shape[1],1).astype('float32')
preds = model.predict(ternary)

se = pd.Series(preds.tolist())
ternary_predictions.insert(loc=0, column='predictions', value=se.values)
#Saving results
ternary_predictions.to_csv('/content/Ternary_Prediction_1DCNN_data_Ternary.csv')

mse = model.evaluate(ternary, y)
print('Mean Squared Error!',mse)


13/13 [==============================] - 0s 11ms/step - loss: 0.0048 - mse: 0.0048 - mae: 0.0578 - root_mean_squared_error: 0.0696
Mean Squared Error! [0.004844578914344311, 0.004844578914344311, 0.057828500866889954, 0.06960301101207733]


In [17]:
#Predicting Tylenol Data
data = pd.read_csv('tylenol_topcare_samples.csv', index_col=0)
data = pd.DataFrame(data)
ternary_predictions = pd.DataFrame()

ternary_predictions["label_name"] =data["label_name"]


data.drop(['label_name'], axis=1, inplace=True)
data.rename(columns={'Unnamed: 0': 'id'},inplace=True)

X = data.values[:,1:]
X = snv(X)
X = pd.DataFrame(X)
#print(X.head(3))

ternary= X.values.reshape(X.shape[0],X.shape[1],1).astype('float32')
preds = model.predict(ternary)

se = pd.Series(preds.tolist())
ternary_predictions.insert(loc=0, column='predictions', value=se.values)
#Saving results
ternary_predictions.to_csv('/content/Tylenol_Prediction_1DCNN.csv')